In [18]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

from time import sleep

In [22]:
# Функция для ожидания кликабельного элемента
def wait_click_func(*args):
    wait = WebDriverWait(driver, timeout=10)
    wait.until(EC.element_to_be_clickable(args)).click()

if __name__ == '__main__':

    url = 'https://apps.sentinel-hub.com/eo-browser'
    
    options = Options()
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", "D:\Github_proj\Brics2024_satellite_fields\data")
    #options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")
    
    driver = webdriver.Firefox(options=options)
    driver.get(url)
    driver.maximize_window()
    
    # Нажатие кнопки использования без авторизации
    wait_click_func(By.XPATH, "//div[@class='eob-btn anon-btn']")
    
    # Нажатие кнопки для пропуска туториала
    wait_click_func(By.XPATH, "//button[@class='tutorial-button tutorial-closebutton']")
    
    # Ввод местоположения откуда хотим снимки
    place = "Moscow"
    driver.find_element(By.XPATH, '//input[@placeholder="Go to Place"]').send_keys(place)
    wait_click_func(By.XPATH, "//div[@class='search-item']")
    
    # Нажатие кнопки поиска снимков
    wait_click_func(By.XPATH, "//div[@class='search-btn-wrapper']")
    #print(driver.find_element(By.XPATH, "//div[@class='tools-container']").get_attribute('innerHTML'))
    #driver.find_element(By.XPATH, "//li[@id='SearchTabButton']").click()
    
    # Нажатие на первый снимок
    sleep(2)
    wait_click_func(By.XPATH, "//div[@id='SearchTab']//div[@class='results']//img[@class='preview-image']")
    
    # Найти все снимки #print(driver.find_element(By.XPATH, "//div[@id='SearchTab']//div[@class='results']//div[@class='results-list']").find_elements(By.CLASS_NAME,'result-item'))
    # Если есть несколько вариантов, нажимаем на первый
    sleep(1)
    if driver.find_elements(By.CLASS_NAME, 'rodal-dialog rodal-slideUp-enter'):
        wait_click_func(By.XPATH, "//div[@class='results-panel']//div[@class='result-item']//img[@class='preview-image']")

    # Кнопка центрирования
    wait_click_func(By.XPATH, "//i[@class='fa fa-crosshairs']")

    #Зум
    zoom = 13
    cur_url = driver.current_url
    # print(cur_url)
    # print(cur_url[cur_url.find('zoom='):cur_url[cur_url.find('zoom='):].find('&')+len(cur_url[:cur_url.find('zoom=')])])
    cur_url = cur_url.replace(cur_url[cur_url.find('zoom='):cur_url[cur_url.find('zoom='):].find('&')+len(cur_url[:cur_url.find('zoom=')])], f'zoom={zoom}')
    # print(cur_url)
    driver.get(cur_url)

    # Скачать
    # Нажатие кнопки использования без авторизации
    wait_click_func(By.XPATH, "//div[@class='eob-btn anon-btn']")
    
    wait_click_func(By.XPATH, "//img[@alt='download-icon']")
    wait_click_func(By.XPATH, "//i[@class='fa fa-download']")
    #<i class="fa fa-download"></i>
    # page = driver.page_source
    # print(page)
    # wait_click_func(By.XPATH, "//path[@class='leaflet-interactive']")

In [ ]:
<a class="eob-btn  full-size "><i class="fa fa-download"></i>Download</a>

In [ ]:
# Можно зарегистрироваться на бесплатный 30 дневный чтобы работать через апишку и токены

In [167]:
import requests

response = requests.post('https://services.sentinel-hub.com/api/v1/process',
  headers={"Authorization" : "Bearer <your_access_token>"},
  json={
    "input": {
        "bounds": {
            "bbox": [
                13.822174072265625,
                45.85080395917834,
                14.55963134765625,
                46.29191774991382
            ]
        },
        "data": [{
            "type": "sentinel-2-l2a"
        }]
    },
    "evalscript": """
    //VERSION=3

    function setup() {
      return {
        input: ["B02", "B03", "B04"],
        output: {
          bands: 3
        }
      };
    }

    function evaluatePixel(
      sample,
      scenes,
      inputMetadata,
      customData,
      outputMetadata
    ) {
      return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
    }
    """
})

In [169]:
response

<Response [401]>